<a href="https://colab.research.google.com/github/smiyawaki0820/pretrained_lm/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Transformers installation
! pip install torch
! pip install transformers
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git


# Training and fine-tuning

このクイックスタートでは、標準的なトレーニングツールを使ってモデルを微調整（またはゼロからトレーニング）する方法をご紹介します。また、複雑なトレーニングを代行するTrainerクラスの使い方もご紹介します。

Overviews:

  - [pytorch](#pytorch)
  - [trainer](#trainer)
  - [additional-resources](#additional-resources)

References:
 - https://huggingface.co/transformers/training.html

<a id='pytorch'></a>

<h2 id='pytorch'>Fine-tuning in native PyTorch</h2>

MLM (BERT) を系列分類のデータセットでファインチューニングを行います。Pytorch を使用したモデルは、[PyTorch Modules](https://pytorch.org/docs/master/generated/torch.nn.Module.html)に従います。


訓練済みモデルのインスタンス化（指定されたモデル構成とその重みを使用して初期化を行う）は、以下のコマンドで行います。タスク特化型の最終層なども含みますが、指定したモデルにない場合は、ランダムな重みで初期化されます。
```python
model = MODEL.from_pretrained(PATH)
# model.eval() 
```

モデルの学習時には、`model.train()` を呼び出します。

In [2]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

任意の最適化を使用することができますが、transformers では AdamW が提供されています。

In [3]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

特定のパラメータ集合に対して異なるハイパラを適用することもできます。例えば、バイアスと layer normalization を除く全てのパラメータに weight decay を適用することができます。

In [5]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

トークナイザを使用して、モデルへの入力テンソルを作成します。

In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text_batch = ["I love Pixar.", "I don't care for Pixar."]
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

分類モデルの呼び出しに `labels` という引数でを使用すると、logits および CE 損失値がリターンされます（`to('cuda')` を使用した GPU 学習も可能です）。


あるいは、`labels` を指定せず、自作した損失関数を使用することも可能です。

In [9]:
import torch

labels = torch.tensor([1,0]).unsqueeze(0)
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
logits = outputs.logits
loss = outputs.loss
loss.backward()
optimizer.step()


""" 自作した損失関数を使用する場合
from torch.nn import functional as F
labels = torch.tensor([1,0])
outputs = model(input_ids, attention_mask=attention_mask)
loss = F.cross_entropy(outputs.logits, labels)
loss.backward()
optimizer.step()
"""

また学習率のスケジューラも用意されています。

以下では、`num_warmup_steps` で warm up し、学習終了時まで 0 に線形減衰するスケジューラを指定します。

使用時には、`optimizer.step()` に続けて `scheduler.step()` を呼びだします。

```python
loss.backward()
optimizer.step()
scheduler.step()
```

In [12]:
from transformers import get_linear_schedule_with_warmup

num_warmup_steps = 1000
num_train_steps = 2000
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)

### Freezing the encoder

パラメータをフリーズしたい場合、エンコーダの `requires_grad` を `False` に設定します。このパラメータには、`base_model` モジュールでアクセスできます。

In [ ]:
for param in model.base_model.parameters():
    param.requires_grad = False

<h2 id='trainer'>Trainer</h2>

`Trainer` を使用することで、訓練、ファインチューニング、推論を始め、ロギングや gradient accumulation、mixed precision など簡単に実行することができます。

`logging_dir` で tensorboard を起動することができます。

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

訓練には、`trainer.train()` を、評価には `trainer.evaluate()` を呼び出します。
独自のモジュールを使用する場合、`forward` から返される第一引数は，最適化したい損失である必要があります。

- バッチ処理を行うために `data_collator`という引数を使って、独自のコレータ関数を渡すことも可能です（データセットから提供されるデータを入力として受け取り、モデルに投入するバッチを返します）。
- 損失に加えて評価基準を追加する場合、独自の `compute_metrics` 関数を定義して、それをトレーナーに渡します。

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

<a id='additional-resources'></a>

<h2 id='additional-resources'>Additional resources</h2>

- [A lightweight colab demo](https://colab.research.google.com/drive/1-JIJlao4dI-Ilww_NnTc0rxtp-ymgDgM?usp=sharing)
  which uses `Trainer` for IMDb sentiment classification.

- [🤗 Transformers Examples](https://github.com/huggingface/transformers/tree/master/examples) including scripts for
  training and fine-tuning on GLUE, SQuAD, and several other tasks.

- [How to train a language model](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb), a detailed
  colab notebook which uses `Trainer` to train a masked language model from scratch on Esperanto.

- [🤗 Transformers Notebooks](https://huggingface.co/transformers/notebooks.html) which contain dozens of example notebooks from the community for
  training and using 🤗 Transformers on a variety of tasks.